# 7. Machine translation and advanced recurrent LSTMs and GRUs

* http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture9.pdf
* http://www.statmt.org/wmt15/translation-task.html
* https://arxiv.org/pdf/1406.1078.pdf

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter, OrderedDict
import nltk
from copy import deepcopy
flatten = lambda l: [item for sublist in l for item in sublist]

In [2]:
USE_CUDA = torch.cuda.is_available()

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [3]:
def getBatch(batch_size,train_data):
    random.shuffle(train_data)
    sindex=0
    eindex=batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex+batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [60]:
def pad_to_batch(batch,x_to_ix,y_to_ix):
    x,y = list(zip(*batch))
    max_x = max([s.size(1) for s in x])
    max_y = max([s.size(1) for s in y])
    x_p,y_p=[],[]
    for i in range(len(batch)):
        if x[i].size(1)<max_x:
            x_p.append(torch.cat([x[i],Variable(LongTensor([x_to_ix['<PAD>']]*(max_x-x[i].size(1)))).view(1,-1)],1))
        else:
            x_p.append(x[i])
        if y[i].size(1)<max_y:
            y_p.append(torch.cat([y[i],Variable(LongTensor([y_to_ix['<PAD>']]*(max_y-y[i].size(1)))).view(1,-1)],1))
        else:
            y_p.append(y[i])
    return list(zip(x_p,y_p))

In [5]:
def prepare_sequence(seq, to_index):
    idxs = list(map(lambda w: to_index[w] if w in to_index.keys() else to_index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

### Data load and Preprocessing 

In [6]:
source_corpus = open('../dataset/europarl-v8.fi-en.en','r',encoding='utf-8').readlines()
target_corpus = open('../dataset/europarl-v8.fi-en.fi','r',encoding='utf-8').readlines()

In [7]:
print(len(source_corpus)==len(target_corpus))
print(len(source_corpus))

True
1926114


In [8]:
from nltk.tokenize import TweetTokenizer
tknz = TweetTokenizer()

In [117]:
parallel_corpus = list(zip(source_corpus,target_corpus))
sampled_parallel_corpus = random.sample(parallel_corpus,100000) # we need to sample data for practice
#sampled_parallel_corpus = parallel_corpus

In [118]:
%%time
X_r,y_r=[],[] # raw

for parallel in sampled_parallel_corpus:
    en,fi = parallel[0][:-1], parallel[1][:-1] # ignore \n
    X_r.append(tknz.tokenize(en))
    y_r.append(tknz.tokenize(fi))

CPU times: user 11 s, sys: 68 ms, total: 11 s
Wall time: 11.1 s


### Build Vocab

In [119]:
source_vocab = list(set(flatten(X_r)))
target_vocab = list(set(flatten(y_r)))

In [120]:
print(len(source_vocab),len(target_vocab))

39462 149580


In paper, they limit the source and target vocabulary to the most frequent 15,000 words

In [156]:
VOCAB_LIMIT = 5000 # this is because we use much smaller dataset. You should determine this hyper-param in practice. 
#We'll deal with this large vocab problem later

In [157]:
source_vocab_count = Counter(flatten(X_r))
source_vocab, _ = list(zip(*source_vocab_count.most_common()[:VOCAB_LIMIT]))

target_vocab_count = Counter(flatten(y_r))
target_vocab, _  = list(zip(*target_vocab_count.most_common()[:VOCAB_LIMIT]))
print(len(source_vocab),len(target_vocab))

5000 5000


In [158]:
source2index = {'<PAD>':0,'<UNK>':1,'<SOS>':2,'<EOS>':3}
for vo in source_vocab:
    source2index[vo]=len(source2index)
index2source = {v:k for k,v in source2index.items()}

target2index = {'<PAD>':0,'<UNK>':1,'<SOS>':2,'<EOS>':3}
for vo in target_vocab:
    target2index[vo]=len(target2index)
index2target = {v:k for k,v in target2index.items()}

In [159]:
%%time
X_p,y_p=[],[]

for so,ta in zip(X_r,y_r):
    X_p.append(prepare_sequence(['<SOS>']+so+['<EOS>'],source2index).view(1,-1))
    y_p.append(prepare_sequence(ta+['<EOS>'],target2index).view(1,-1))
    
data_p = list(zip(X_p,y_p))
random.shuffle(data_p)
train_data = data_p[:int(len(data_p)*0.9)]
test_data = data_p[int(len(data_p)*0.9):]

CPU times: user 5.69 s, sys: 44 ms, total: 5.74 s
Wall time: 5.74 s


### Modeling 

In [162]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size,hidden_size, n_layers=1):
        super(Encoder, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True)
    
    def init_hidden(self,inputs):
        hidden = Variable(torch.zeros(self.n_layers,inputs.size(0),self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden
    
    def init_weight(self):
        self.embedding.weight = nn.init.xavier_normal(self.embedding.weight)
        self.gru.weight_hh_l0 = nn.init.xavier_normal(self.gru.weight_hh_l0)
        self.gru.weight_ih_l0 = nn.init.xavier_normal(self.gru.weight_ih_l0)
    
    def forward(self, inputs, input_masking):
        """
        inputs : B,T (LongTensor)
        input_masking : B,T (ByteTensor)
        """
        hidden = self.init_hidden(inputs)
        
        embedded = self.embedding(inputs)
        output, hidden = self.gru(embedded, hidden)

        real_context=[]
        
        for i,o in enumerate(output): # B,T,D
            real_length = input_masking[i].data.tolist().count(0) # real length
            real_context.append(o[real_length-1])
            
        return output, torch.cat(real_context).view(inputs.size(0),-1).unsqueeze(1)

In [163]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, n_layers=1,dropout_p=0.3):
        super(Decoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        # Define the layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.dropout = nn.Dropout(dropout_p)
        
        self.gru = nn.GRU(embedding_size+hidden_size, hidden_size, n_layers,batch_first=True)
        self.linear = nn.Linear(embedding_size+hidden_size*2, input_size)
    
    def init_hidden(self,inputs):
        hidden = Variable(torch.zeros(self.n_layers,inputs.size(0),self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden
    
    
    def init_weight(self):
        self.embedding.weight = nn.init.xavier_normal(self.embedding.weight)
        self.gru.weight_hh_l0 = nn.init.xavier_normal(self.gru.weight_hh_l0)
        self.gru.weight_ih_l0 = nn.init.xavier_normal(self.gru.weight_ih_l0)
        self.linear.weight = nn.init.xavier_normal(self.linear.weight)
    
    def forward(self, inputs, context,max_length,training=False):
        """
        inputs : B,1 (LongTensor, START SYMBOL)
        context : B,1,H (FloatTensor, Last encoder hidden state)
        max_length : int, max length to decode
        training : bool, this is because adapt dropout only training step.
        """
        # Get the embedding of the current input word
        embedded = self.embedding(inputs)
        hidden = self.init_hidden(inputs)
        if training:
            embedded = self.dropout(embedded)
        
        decode=[]
        # Apply GRU to the output so far
        for i in range(max_length):

            _, hidden = self.gru(torch.cat((embedded,context),2), hidden) # h_t = f(h_{t-1},y_{t-1},c)
            concated = torch.cat((hidden,embedded.transpose(0,1),context.transpose(0,1)),2) # y_t = g(h_t,y_{t-1},c)
            score = self.linear(concated.squeeze(0))
            softmaxed = F.log_softmax(score)
            decode.append(softmaxed)
            decoded = softmaxed.max(1)[1]
            embedded = self.embedding(decoded).unsqueeze(1) # y_{t-1}
            if training:
                embedded = self.dropout(embedded)
            
        #  column-wise concat, reshape!!
        scores = torch.cat(decode,1)
        return scores.view(inputs.size(0)*max_length,-1)

### Train 

It takes for a while

In [164]:
STEP=10
BATCH_SIZE = 32
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 512
LR = 1e-3

In [ ]:
encoder = Encoder(len(source2index),EMBEDDING_SIZE,HIDDEN_SIZE)
decoder = Decoder(len(target2index),EMBEDDING_SIZE,HIDDEN_SIZE)
encoder.init_weight()
decoder.init_weight()

if USE_CUDA:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

loss_function = nn.CrossEntropyLoss(ignore_index=0)
enc_optimizer = optim.Adam(encoder.parameters(),lr=LR)
dec_optimizer = optim.Adam(decoder.parameters(),lr=LR)

In [ ]:
for step in range(STEP):
    losses=[]
    for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
        batch = pad_to_batch(batch,source2index,target2index)
        x,y = list(zip(*batch))
        inputs = torch.cat(x)
        targets = torch.cat(y)
        
        input_mask = torch.cat([Variable(ByteTensor(tuple(map(lambda s: s ==0, t.data)))) for t in inputs]).view(inputs.size(0),-1)
        start_decode = Variable(LongTensor([[target2index['<SOS>']]*targets.size(0)])).transpose(0,1)
        
        encoder.zero_grad()
        decoder.zero_grad()
        output, hidden_c = encoder(inputs,input_mask)
        
        preds = decoder(start_decode,hidden_c,targets.size(1))
                                
        loss = loss_function(preds,targets.view(-1))
        losses.append(loss.data.cpu().numpy()[0] if USE_CUDA else loss.data.numpy()[0] )
        loss.backward()
        torch.nn.utils.clip_grad_norm(encoder.parameters(), 0.5) # gradient clipping
        torch.nn.utils.clip_grad_norm(decoder.parameters(), 0.5) # gradient clipping
        enc_optimizer.step()
        dec_optimizer.step()
        
        if i % 1000==0:
            print("[%d/%d] mean_loss : %0.2f" %(step,STEP,np.mean(losses)))
            losses=[]

[0/10] mean_loss : 8.51


### Test 

BLEU